In [3]:
#importing the necessary libraries
import cv2
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [4]:
#extracting the names and location of the train videos
with open('trainlist01.txt') as file1 :
    trainData = file1.read()
    trainData = trainData.split('\n')

    
#extracting the names and location of the test videos
with open('testlist01.txt') as file1 :
    testData = file1.read()
    testData = testData.split('\n')

    
#extracting the classes and their indices   
with open('classInd.txt') as file1 :
    classInd = file1.read()
    classInd = classInd.split('\n')

In [53]:
#making dictionary to assign a class index for every class name
class_to_Ind = {}

#looping over the classes to extract the name and index then stroing them in the dictionary
for item in classInd[:-1]:
    
    ind , className = item.split()
    
    class_to_Ind[className.lower()] = int(ind)

In [5]:
#creating a data frame to hold the name and the class of the train data
videos_train = pd.DataFrame()
videos_train['nameAndClass'] = trainData[:-1]

#creating a data frame to hold the name and the class of the test data
videos_test = pd.DataFrame()
videos_test['name'] = testData[:-1]

In [6]:
#the number of videos in both , the train and test data
len(testData) + len(trainData) - 2

13320

In [7]:
#extracting the names of the videos
videos_train['name'] = videos_train['nameAndClass'].apply(lambda x : x.split()[0])

#extracting the classes of the videos
videos_train['class'] = videos_train['nameAndClass'].apply(lambda x : x.split()[1])

In [8]:
videos_test.head(10)

,name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi
5,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c06.avi
6,ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c01.avi
7,ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c02.avi
8,ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c03.avi
9,ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c04.avi


In [9]:
videos_train.head(10)

,nameAndClass,name,class
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi,1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi,1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi,1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi,1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi,1
5,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c01.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c01.avi,1
6,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c02.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c02.avi,1
7,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c03.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c03.avi,1
8,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c04.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c04.avi,1
9,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c05.avi 1,ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c05.avi,1


In [17]:
def saveFrames (wantedFPS  , VidDir , VidName ,  saveDir ):
    """
    Saves the frames of a given video at the given location and returns the number of frames saved.

    Parameters
    ----------
    wantedFPS : int
        The number of frames per second to save the frames with.
    
    VidDir : string
        the direction of the video that needs to be processed.
        
    VidName : string
        the name of the video that needs to be processed.
        
    saveDir : string
        the desired direction to save the frames (make sure it exists)

    Returns
    -------
    the number of frames saved in the desk.
    
    """
    
    VidDirPlusName = VidDir + VidName
    Vid = cv2.VideoCapture(VidDirPlusName)
    fps = Vid.get(cv2.CAP_PROP_FPS) #the real fps of the chosen video
    
    #making sure that the wantedFPS smaller than the real fps
    assert fps >= wantedFPS , "the wantedFPS should be smaller than the real fps"
    
    #every time the counter can be divided by the checkmark, a frame will be saved to get the wantedFPS
    checkmark = math.floor(fps / wantedFPS) 
    frame_counter = 0
    counter = 0
    
    while Vid.isOpened():
    
        ret , frame = Vid.read()

        if ret == False : #if there're no more frames in the video, the loop will break        
            break

        counter += 1
        if counter % checkmark == 0 :
            frame_counter += 1
            framename = saveDir + VidName + '_' + str(frame_counter) + '.jpg'

            cv2.imwrite(framename , frame)


    Vid.release()
    return frame_counter

In [27]:
#extracting the frames of the train videos

videos_train['fps'] = 0 #intializing a column of zeroes that will hold the number of stored frames of each video after processing
wantedFPS = 5 #the chosen number of frames per second 

for i in range(len(videos_train)):
    
    VidDir = 'UCF-101/' + videos_train['name'][i].split('/')[0] + '/'
    VidName = videos_train['name'][i].split('/')[1]
    saveDir = 'Train_Frames_1/'
    
    #saving the frames to the output direction then storing the number of frames of that video
    videos_train['fps'][i] = saveFrames (wantedFPS  , VidDir , VidName ,  saveDir )
    
    #viewing the progress precentage
    clear_output()
    print(str( (100 * (i+1) ) / len(videos_train)) + ' %')

100 %


In [28]:
#extracting the frames of the test videos

videos_test['fps'] = 0 #intializing a column of zeroes that will hold the number of stored frames of each video after processing
wantedFPS = 5 #the chosen number of frames per second 

for i in range(len(videos_test)):
    
    VidDir = 'UCF-101/' + videos_test['name'][i].split('/')[0] + '/'
    VidName = videos_test['name'][i].split('/')[1]
    saveDir = 'Test_Frames_1/'
    
    #saving the frames to the output direction then storing the number of frames of that video
    videos_test['fps'][i] = saveFrames (wantedFPS  , VidDir , VidName ,  saveDir ) 
    
    #viewing the progress precentage
    clear_output()
    print(str( (100 * (i+1) ) / len(videos_test)) + ' %')

100 %


In [31]:
#making a list of the frames names and thier classes in order
frames_train = []

#looping over the names 
for i  in range( len( videos_train ) ):
    
    VidName = videos_train['name'][i].split('/')[1] #the name of the video
    VidClass = videos_train['class'][i] #the class of the video
    VidFrames = videos_train['fps'][i] #the number of stored frames of the video
    
    #saving the frame names and classes in tuple (frame_name , class)
    frames_train+= [(VidName + '_' + str(j) + '.jpg' , VidClass ) for j in range ( 1 , VidFrames + 1) ]
        

#creating a data frame to hold the frames names and classes
frames_train = pd.DataFrame(frames_train , columns = ['name' , 'class'])
frames_train.to_csv('frames_train_1.csv' , index=False)

In [55]:
#making a list of the frames names and thier classes in order
frames_test = []

#looping over the names 
for i  in range( len( videos_test ) ):
    
    VidName = videos_test['name'][i].split('/')[1] #the name of the video
    ClassName = videos_test['name'][i].split('_')[1].lower() #the class name
    VidClass = class_to_Ind[ ClassName ] #the index of the class using class_to_Ind
    VidFrames = videos_test['fps'][i] #the number of stored frames of the video
    
    #saving the frame names and classes in tuple (frame_name , class)
    frames_test += [(VidName + '_' + str(j) + '.jpg' , VidClass ) for j in range ( 1 , VidFrames + 1) ]
        

#creating a data frame to hold the frames names and classes
frames_test = pd.DataFrame(frames_test , columns = ['name' , 'class'])
frames_test.to_csv('frames_test_1.csv' , index=False)